<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/llama_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install sentencepiece
!pip install git+https://github.com/huggingface/transformers
!pip install rich[jupyter]
!pip install deepspeed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_6sqry06
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_6sqry06
  Resolved https://github.com/huggingface/transformers to commit 5990743fddb4780b15b8af2ed7ab55145ab40455
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6758236 sha256=0a02

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.4/765.4 KB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 20.8 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.8.3-py3-none-any.whl size=776415 sha256=32e201b247eeb4ce421f5ff59fed9f54ecb98372f7d399341cc4052fff705f39
  Stored in directory: /root/.cache/pip/wheels/f8/ea/8f/0768328ba436ed66f602d8d3b809624448c9eb627434176d04
Successfully built deepspeed


In [ ]:
Llama_Weights = '/content/drive/MyDrive/experiments/llama/LLaMA/7B/'
Output = 'drive/MyDrive/experiments/HF_Llama'

In [ ]:
#!python /content/drive/MyDrive/experiments/llama/convert_llama_weights_to_hf.py \
#--input_dir '/content/drive/MyDrive/experiments/llama/LLaMA/7B' \
# --model_size 7B --output_dir 'drive/MyDrive/experiments/HF_Llama'

##Vanilla Llama##

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, LlamaTokenizer, LlamaForCausalLM


model = LlamaForCausalLM.from_pretrained('sallywww/Llama-7B')
tokenizer = AutoTokenizer.from_pretrained('sallywww/Llama-7B')

#model.push_to_hub('sallywww/Llama-7B')
#tokenizer.push_to_hub('sallywww/Llama-7B')

In [ ]:
assert(False)
prompt = "a loop invariant in computer science is "
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=40)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


AssertionError: ignored

##loading and json-ize data##

In [4]:
!pip install simplejson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.8/136.8 KB 15.4 MB/s eta 0:00:00


In [7]:
from collections import OrderedDict
import simplejson as json
from numpy import random
import pandas as pd
import io
import os
import pandas
import xlrd
import json
import xlrd

#workbook = xlrd.open_workbook('/invariants_line_number.xlsx', on_demand = True)
df = pandas.read_excel(open('/content/invariants_data.xlsx','rb'))
df

,Source,Target
0,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 contra...,14+ assert (y == x + 4);
1,1 // SPDX-License-Identifier: MIT\n2 pragma ex...,"109+ require(admin == msg.sender, ""Ownable: c..."
2,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 contra...,7+ assert (y == x + 2);
3,1 pragma solidity >=0.4.24 <0.6.0;\n2 contract...,20+ assert(funcA2(funcA1())==12);\n20+ assert(...
4,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 // Thi...,24+ assert(a == x + 1);\n32+ assert(a == x);\n...
...,...,...
98,1 pragma solidity >=0.4.24 <0.6.0;\n2 \n3 cont...,6+ assert (!false);\n6+ assert (!false);
99,1 // SPDX-License-Identifier: MIT\n2 pragma ...,6+ assert(fee + value != 0);\n6+ assert(fee ...
100,1 // SPDX-License-Identifier: MIT\n2 pragma ...,15+ assert(_required > 0); \n15+ assert(m_num...
101,1 / SPDX-License-Identifier: MIT\n2 pragma so...,6+ assert(address(this)==msg.sender);\n6+ as...


In [8]:
def _make_w_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f_dirname = os.path.dirname(f)
        if f_dirname != "":
            os.makedirs(f_dirname, exist_ok=True)
        f = open(f, mode=mode)
    return f


def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f


def jdump(obj, f, mode="w", indent=4, default=str):
    """Dump a str or dictionary to a file in json format.
    Args:
        obj: An object to be written.
        f: A string path to the location on disk.
        mode: Mode for opening the file.
        indent: Indent for storing json dictionaries.
        default: A function to handle non-serializable entries; defaults to `str`.
    """
    f = _make_w_io_base(f, mode)
    if isinstance(obj, (dict, list)):
        json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str):
        f.write(obj)
    else:
        raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()


def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict


In [10]:
# List to hold dictionaries
def prepare_actor_model_data(df): 
    data_list=[]
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data["completion"] = row['Target']
        data_list.append(data)
    jdump(data_list, 'datasets/actor_training_data.json')

def prepare_rlhf_model_data(df): 
    data_list=[]
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data_list.append(data)
    jdump(data, '/datasets/reward_training_data.json')


def prepare_reward_model_data(df):
    data_list=[]    
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data["completion"] = row['Target']
        data["score"] = 5
        data_list.append(data)
    jdump(data, '/content/datasets/rlhf_training_data.json')


In [11]:
prepare_actor_model_data(df)
prepare_rlhf_model_data(df)
prepare_reward_model_data(df)

In [ ]:
#from google.colab import files
#!zip -r /content/datasets.zip /content/datasets
#files.download("/content/datasets.zip")

In [ ]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: tokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)


def _tokenize_fn(strings: Sequence[str], tokenizer: tokenizer) -> Dict:   
    """Tokenize a list of strings."""
    
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [12]:
if tokenizer.pad_token is None or tokenizer.eos_token:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  
print(tokenizer.pad_token)

Using pad_token, but it is not set yet.


[PAD]


In [ ]:
with open('/content/datasets/actor_training_data.json', "r") as f:
    data = json.load(f)
    for d in data:
     item = [str(d["user_input"]) + "\n" + str(d["completion"]) for d in data]
    length = len(item)
sources = [f"{example['user_input']}{tokenizer.eos_token}" for example in data]#
targets = [f"{example['completion']}{tokenizer.eos_token}" for example in data]

data_dict = preprocess(sources, targets, tokenizer)
input_ids = data_dict["input_ids"]
labels = data_dict["labels"]

##Finetuning using stanford dataprocessing## 

In [13]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 33.9 MB/s eta 0:00:00


In [14]:
data_path ='/content/datasets/actor_training_data.json'

In [16]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence

import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer
import dataclasses
import logging
import math
import os
import io
import sys
import time
import json
from typing import Optional, Sequence, Union

import openai
import tqdm
from openai import openai_object
import copy

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"

def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict

def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: tokenizer,
    model: model,
):
    """Resize tokenizer and embedding.
    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg


def _tokenize_fn(strings: Sequence[str], tokenizer: tokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: tokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)


class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""
    def __init__(self, data_path: str, tokenizer: tokenizer) -> None:
        logging.warning("Loading data...")
        self.data_path = data_path
        with open(data_path, "r") as f:
            data = json.load(f)
            self.data =  [str(d["user_input"]) + "\n" + str(d["completion"]) for d in data]
        self.len = len(self.data)
        sources = [f"{example['user_input']}{tokenizer.eos_token}" for example in data]
        targets = [f"{example['completion']}{tokenizer.eos_token}" for example in data]
        data_dict = preprocess(sources, targets, tokenizer)
        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
    
    def __len__(self):
        #print(f"comparing length with chatlllama definition: chatllama: {self.data[idx]} v.s. stanford version{self.input_ids}")
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""
    def __init__(self, tokenizer: tokenizer) -> None:
        logging.warning("data collator for supervised learning...")
        self.tokenizer=tokenizer
    
    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

def make_supervised_data_module(tokenizer: tokenizer, data_path) -> Dict:
    """Make dataset and collator for supervised fine-tuning."""
    train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path=data_path)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments("working_dir")
training_args = training_args.set_lr_scheduler(name="cosine", warmup_ratio=0.05)
training_args.fp16 = True
training_args.auto_find_batch_size=False
training_args.per_device_train_batch_size=1
training_args.per_device_train_batch_size=1
training_args.output_dir='./output'
training_args.num_train_epochs = 3 
training_args.gradient_accumulation_steps = 8 
training_args.evaluation_strategy="no" 
training_args.save_strategy="steps" 
training_args.save_steps=2000 
training_args.save_total_limit=1 
training_args.learning_rate=2e-5 
training_args.weight_decay=0. 
training_args.warmup_ratio=0.03 
training_args.lr_scheduler_type="cosine" 
training_args.logging_steps=1  
training_args.fsdp="full_shard auto_wrap"
training_args.fsdp_transformer_layer_cls_to_wrap='LLaMADecoderLayer' 
training_args.tf32=True
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore

In [18]:
import functools
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp.fully_sharded_data_parallel import (
    CPUOffload,
    BackwardPrefetch,
)
from torch.distributed.fsdp.wrap import (
    size_based_auto_wrap_policy,
    enable_wrap,
    wrap,
)

def train(rank, model, tokenizer):
    if tokenizer.pad_token is None:
        smart_tokenizer_and_embedding_resize(
            special_tokens_dict=dict(pad_token=DEFAULT_PAD_TOKEN),
            tokenizer=tokenizer,
            model=model,
        )

    tokenizer.add_special_tokens(
        {
             "eos_token": DEFAULT_EOS_TOKEN,
             "bos_token": DEFAULT_BOS_TOKEN,
             "unk_token": DEFAULT_UNK_TOKEN,
         }
     )
    data_module = make_supervised_data_module(tokenizer=tokenizer, data_path=data_path)
    my_auto_wrap_policy = functools.partial(
        size_based_auto_wrap_policy, min_num_params=100
    )
    torch.cuda.set_device(rank)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=200, num_training_steps=-1
    )
    train_loader = DataLoader(data_module, batch_size=1, shuffle=True)
    model = model.to(rank)
    model = FSDP(model, fsdp_auto_wrap_policy=my_auto_wrap_policy)
    ddp_loss = torch.zeros(2).to(rank)
    for epoch in range(3):
        for batch in train_loader:
            optim.zero_grad()
            input_ids = batch['input_ids'].to(rank)
            attention_mask = batch['attention_mask'].to(rank)
            labels = batch['labels'].to(rank)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            loss.backward()
            optim.step()
            ddp_loss[0] += loss.item()
            ddp_loss[1] += len(data)
            torch.cuda.empty_cache()

    dist.all_reduce(ddp_loss, op=dist.ReduceOp.SUM)
    if rank == 0:
        print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, ddp_loss[0] / ddp_loss[1]))



In [26]:
train(0, model, tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
os.environ['CUDA_VISIBLE_DEVICES']='0, 1, 2, 3, 4, 5, 6, 7, 8'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
#train(model, tokenizer)

In [ ]:
!pip install libnvinfer_plugin.so.8

In [ ]:
!torchrun --nproc_per_node 1 /content/drive/MyDrive/train.py 

In [ ]:
!deepspeed --num_gpus=0 /content/drive/MyDrive/train.py --deepspeed /content/drive/MyDrive/artifacts/config/ds_config.json

[2023-03-21 08:31:54,168] [WARNING] [runner.py:186:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-03-21 08:31:54,765] [INFO] [runner.py:550:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None /content/drive/MyDrive/train.py --deepspeed /content/drive/MyDrive/artifacts/config/ds_config.json
[2023-03-21 08:31:57,334] [INFO] [launch.py:135:main] 0 NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.16.2-1+cuda11.8
[2023-03-21 08:31:57,334] [INFO] [launch.py:135:main] 0 NV_LIBNCCL_DEV_PACKAGE_VERSION=2.16.2-1
[2023-03-21 08:31:57,334] [INFO] [launch.py:135:main] 0 NCCL_VERSION=2.16.2-1
[2023-03-21 08:31:57,334] [INFO] [launch.py:135:main] 0 NV_LIBNCCL_DEV_PACKAGE_NAME=libnccl-dev
[2023-03-21 08:31:57,334] [INFO] [launch.py:135:main] 0 NV_LIBNCCL_PACKAGE=libnccl2=2.16.2-1+cuda11.8
[2023-03-21 08:31:57,334] [INFO] [launch.py:135:main] 0

In [ ]:
assert(False)
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence

import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer
import dataclasses
import logging
import math
import os
import io
import sys
import time
import json
from typing import Optional, Sequence, Union

import openai
import tqdm
from openai import openai_object
import copy

#import utils

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = [{
    "user_input":"solidity program",
    "completion": "invariants"}]

'''
@dataclass
class ModelArguments:
    model_name_or_path:Optional[str] = field(default='/content/drive/MyDrive/experiments/llama/LLaMA/7B')


@dataclass
class DataArguments:
    data_path: str = field(default=None, metadata={"help": "/content/datasets"})


@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=512,
        metadata={"help": "Maximum sequence length. Sequences will be right padded (and possibly truncated)."},
    )
'''
def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f

def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict

def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """Collects the state dict and dump to disk."""
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save:
        cpu_state_dict = {key: value.cpu() for key, value in state_dict.items()}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)  # noqa


def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.
    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg


def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)


class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = jload(data_path)

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["user_input"], PROMPT_DICT["completion"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

'''
def make_supervised_data_module(tokenizer: transformers.PreTrainedTokenizer, data_args) -> Dict:
    """Make dataset and collator for supervised fine-tuning."""
    train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path=data_args.data_path)
    data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
    return dict(train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)


def train():
    parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
    )

    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=False,
    )
    if tokenizer.pad_token is None:
        smart_tokenizer_and_embedding_resize(
            special_tokens_dict=dict(pad_token=DEFAULT_PAD_TOKEN),
            tokenizer=tokenizer,
            model=model,
        )
    if "llama" in model_args.model_name_or_path:
        tokenizer.add_special_tokens(
            {
                "eos_token": DEFAULT_EOS_TOKEN,
                "bos_token": DEFAULT_BOS_TOKEN,
                "unk_token": DEFAULT_UNK_TOKEN,
            }
        )

    data_module = make_supervised_data_module(tokenizer=tokenizer, data_args=data_args)
    trainer = Trainer(model=model, tokenizer=tokenizer, args=training_args, **data_module)
    trainer.train()
    trainer.save_state()
    safe_save_model_for_hf_trainer(trainer=trainer, output_dir='/content/saved_model')
 

#if __name__ == "__main__":
#    train()

In [20]:
!git clone https://github.com/tatsu-lab/stanford_alpaca.git

Cloning into 'stanford_alpaca'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 100 (delta 44), reused 38 (delta 38), pack-reused 41
Receiving objects: 100% (100/100), 9.13 MiB | 5.43 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [24]:
!pip install --upgrade torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 32.1 MB/s eta 0:00:00
     ━━━━━

In [23]:
!torchrun --nproc_per_node=1 --master_port=22 /content/stanford_alpaca/train.py \
    --model_name_or_path '/content/drive/MyDrive/experiments/HF_Llama/llama-7b'\
    --data_path '/content/datasets/actor_training_data.json' \
    --bf16 True \
    --output_dir '/content/saved_model' \
    --num_train_epochs 3 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 8 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 2000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --fsdp "full_shard auto_wrap" \
    --fsdp_transformer_layer_cls_to_wrap 'LLaMADecoderLayer' \
    --tf32 True

2023-03-21 23:26:16.854166: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-21 23:26:16.854251: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-21 23:26:16.854263: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent call last):
  File "/content/stanford_alpaca/train.py", line 231, in <module>
    train()
  File "/content/stanford_alpaca/train.py", line 194, 

##Finetuning##

In [ ]:
!pip install chatllama-py

In [ ]:
#!zip -r /content/raw_datasets.zip /content/datasets
#from google.colab import files
#files.download("/content/raw_datasets.zip")

In [ ]:
!pip install deepspeed
!pip install mpi4py
!pip install ninja

In [ ]:
#!wget -O artifacts.zip https://nbllabartifacts.blob.core.windows.net/chatllama/artifacts.zip\?sp\=r\&st\=2023-03-08T14:53:24Z\&se\=2100-03-08T22:53:24Z\&spr\=https\&sv\=2021-06-08\&sr\=b\&sig\=jqr%2B2ZkR0SW9RjV0pDOdQ%2BDulLXLjbZ36vmNd4XxxyQ%3D
#!unzip artifacts.zip


In [ ]:
!git clone https://github.com/facebookresearch/llama.git
!pip install -r /content/llama/requirements.txt
!pip install -e /content/llama/.

In [ ]:
!deepspeed /content/artifacts/main.py -t ACTOR /content/artifacts/config/config.yaml

##GPT-style finetuning: OOM error persisted##

In [ ]:
from google.colab import auth
from google.auth import default
from numpy import random
import pandas as pd
import gspread
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
#defining my worksheet
worksheet = gc.open('short_invariants_for_gpt').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
cols = ['Target']
df = pd.DataFrame(rows, columns=cols)
df

In [ ]:
model_params={
    "MODEL":"llama_model",             # model_type: t5-large
    "MAX_LENGTH": 1024,  # max length of source text
   # "SEED": random.randint(1000)    # randomized seeds to shuffle test set

}

In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from rich.console import Console
from tqdm import tqdm, trange
import torch.nn.functional as F
import random
import torch
import csv

class GPTDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, target_label, truncate=False, \
               gpt2_type=model, \
               max_length=2048):
    self.tokenizer = tokenizer
    self.target_tokens = []

    for row in df['Target']:
        self.target_tokens.append(torch.tensor(
            self.tokenizer.encode(f"<|{target_label}|>{row[:max_length]}<|endoftext|>")
            ))  
    if truncate:
            self.target_tokens = self.target_tokens[:20000]
    self.length = len(self.target_tokens)   

  def __len__(self):
    return self.length

  def __getitem__(self, index):
    return self.target_tokens[index]   

In [ ]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=0.1, epochs=3, lr=2e-5,
    max_seq_len=2048, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False, save_model_on_epoch=False,
):
    acc_steps = 10
    device=torch.device("gpu")
    model = model.to_device('gpu')
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None
    loss_list = []
    epoch_list = []
    for epoch in range(epochs):
        total_loss = []
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to('gpu')
            outputs = model(input_tensor, labels=input_tensor)  
            loss = outputs[0] 
            total_loss.append(float(loss.item()))       
            loss.backward()
            torch.cuda.empty_cache()                      
            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()
  
            accumulating_batch_count += 1
            input_tensor = None

        #training_logger.add_row(str(epoch), str(np.mean(total_loss)))       
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
        loss_list.append(np.mean(total_loss))
        epoch_list.append(epoch) 
        print(f"for epoch {epoch} the loss is {np.mean(total_loss)}\n")
   # console.print(training_logger)   
   # plot_loss(epoch_list, loss_list)
    return model

In [ ]:
def fine_tune_GPT2(df, model_params):   
  #console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  dataset = GPTDataSetClass(df['Target'], truncate=False, gpt2_type=model) 
  #Get the tokenizer and model
  #tokenizer = GPT2Tokenizer.from_pretrained(model_params["MODEL"])
 # model = GPT2LMHeadModel.from_pretrained(model_params["MODEL"])  
  trained_model = train(dataset, model, tokenizer)
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join('/content/output', "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)
  console.print(f"""[Model] Model saved @ {os.path.join('/content/output', "model_files")}\n""")
  
  # logging
  console.log(f"[Data]: Reading Raw data...\n")


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 

  console.print(f"FULL Dataset: {df.shape}")
  return trained_model, tokenizer


In [ ]:

def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=2048, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text + '\n')
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text + '\n')
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(model, tokenizer, test_data):
  generated_code = []
  for i in range(len(test_data)):    
    x = generate(model.to('cpu'), tokenizer, test_data['Target'][i], entry_count=1)
    generated_code.append(x)
  return generated_code

#Run the functions to generate the lyrics

def test_fine_tuned_gpt2(model, tokenizer, df): 
    test_set = df.sample(n=1)
    df = df.loc[~df.index.isin(test_set.index)]

    #Reset the indexes
    test_set = test_set.reset_index()
    df = df.reset_index()

    #For the test set only, keep last 20 words in a new column, then remove them from original column
    test_set['Target'] = test_set['Target'].str.split().apply(' '.join)
    generated_code = text_generation(model, tokenizer, test_set)
    print(generated_code)


In [ ]:
#To solve CUDA out of memory error
import gc
gc.collect()
torch.cuda.empty_cache()
os.environ['CUDA_VISIBLE_DEVICES']='0, 1, 2, 3, 4, 5, 6, 7, 8'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
trained_model, trained_tokenizer = fine_tune_GPT2(df, model_params)

In [ ]:
import json
import os

import deepspeed
import torch
from beartype import beartype
from beartype.typing import Tuple
from einops import rearrange
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)



  
local_rank, world_size = setup_model_parallel()
            # use load_model_test for testing
  self.model, self.tokenizer = load_model(
   ckpt_dir=model,
   tokenizer_path=tokenizer,
   local_rank=local_rank,
   world_size=world_size,
   froze_embeddings=config.froze_embeddings,
   use_fairscale=config.use_fairscale,
   max_batch_size=config.batch_size,
    )
        elif config.model in hf_models_seq_2_seq:
    
            # galactica tokenizer eos_token is None
            if self.tokenizer.eos_token is None:
                self.tokenizer.eos_token = "</s>"
                self.tokenizer.eos_token_id = 0
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
            self.model = AutoModelForCausalLM.from_pretrained(
                config.model,
            )
            self.model.to(config.device)

        # load the model from model_folder
        self.load()

    @beartype
    def load(self) -> None:
        """Load the model from the path

        Args:
            path (str): path to the model
        """
        path = ModelLoader.check_model_path(
            config=self.config,
            is_checkpoint=False,
            current_epoch=None,
        )
        if path is not None:
            print("Loading ...")
            model_dict = torch.load(path)
            self.model.load_state_dict(model_dict["model"])
            self.head.load_state_dict(model_dict["head"])

    @beartype
    def save(self) -> None:
        """Save the model to the path

        Args:
            path (Optional[str], optional): Path to store the model.
                Defaults to None.
        """
        model_folder, model_name, path = ModelLoader.get_model_path(
            config=self.config,
            is_checkpoint=False,
            current_epoch=None,
        )
        print(f"Saving model to {path} ...")
        torch.save(
            {"model": self.model.state_dict(), "head": self.head.state_dict()},
            path,
        )

    def parameters(self, **kwargs):
        """Return the parameters of the model

        Args:
            **kwargs:
        """
        return self.model.parameters()

    @beartype
    def forward(
        self, sequences: torch.Tensor, sequences_mask: torch.Tensor
    ) -> torch.Tensor:
        """Generate logits to have probability distribution over the vocabulary
            of the actions

        Args:
            sequences (torch.Tensor): Sequences of states and actions used to
                    compute token logits for the whole list of sequences
            attention_mask (torch.Tensor): Mask for the sequences attention

        Returns:
            logits (torch.Tensor): Logits for the actions taken
        """
        model_output = self.model.forward(
            sequences, attention_mask=sequences_mask
        )
        # need to return logits for the actions
        if self.config.model in hf_models_causal_lm:
            model_output = model_output.logits
        if self.config.debug:
            print("ActorModel.forward")
            print("model_output_logits shape", model_output.shape)
            print("model_output logits", model_output)
        return model_output

    @beartype
    @torch.no_grad()
    def generate(
        self, states: torch.Tensor, state_mask: torch.Tensor
    ) -> Tuple:
        """Generate actions and sequences=[states, actions] from state
            (i.e. input of the prompt generator model)

        Args:
            state (torch.Tensor): the input of the user
            state_mask (torch.Tensor): Mask for the state input (for padding)

        Returns:
            actions (torch.Tensor): Actions generated from the state
            sequences (torch.Tensor): Sequences generated from the
                state as [states, actions]
        """
        temperature = self.config.temperature
        # max sequence length for the actor (i.e. prompt + completion)
        # from config file - it depends by the model used
        max_sequence_length = self.config.max_sequence_length
        # max tokens generated by the actor (completion only) from config file
        max_tokens = self.config.max_tokens
        # temperature for the actor
        max_generation_possible = max_sequence_length - states.shape[1]
        # take the minimum between the maximum token that you want to generate
        # and the token that is possible to generate given the maximum sequence
        # supported
        max_completion = min(max_tokens, max_generation_possible)
        if max_completion <= 0:
            raise ValueError(
                "The maximum completion available is <= 0 the prompt is too "
                + "long w.r.t the model sequence length"
            )
        # the max_length is then the input length + the completion length
        max_length = states.shape[1] + max_completion
        # generate
        sequences = self.model.generate(
            input_ids=states,
            attention_mask=state_mask,
            temperature=temperature,
            max_length=max_length,
        )
        actions = sequences[:, states.shape[1] :]  # noqa E203
        if self.config.debug:
            print("ActorModel.generate")
            print("state", states)
            print("state shape", states.shape)
            print("sequence shape", sequences.shape)
            print("sequence", sequences)
            print("actions shape", actions.shape)
            print("actions", actions)
        return actions, sequences


class ActorDataset(Dataset):
    """Dataset for the pretraining of the actor model
    read a json file with the following format:
    [
        {
            "user_input": "..."
            "completion": "..."
        },
        ...
    ]
    Where:
        user_input: the input of the user
        completion: the output of the user
    """

    def __init__(self, path: str) -> None:
        self.path = path
        with open(path, "r") as f:
            data = json.load(f)
            self.data = [d["user_input"] + " " + d["completion"] for d in data]
        self.len = len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(
        self,
    ):
        return self.len


class ActorTrainer:
    """Used to pre-train the actor model to generate better prompts.

    Args:
        config (ConfigActor): Configuration for the actor model

    Attributes:
        config (ConfigActor): Configuration for the actor model
        model (ActorModel): Actor model
        loss_function (torch.nn.CrossEntropyLoss): Loss function
        optimizer (torch.optim.Adam): Optimizer
        validation_flag (bool): Flag to indicate if the validation dataset
            is provided
        training_stats (TrainingStats): Training statistics

    Methods:
        train: Train the actor model
    """

    def __init__(self, config: ConfigActor) -> None:
        # load the model, optimizer, loss function and config
        self.config = config
        self.model = ActorModel(config)
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=config.lr
        )

        # check checkpoint, datasets and other data
        self.validation_flag = False
        self.training_stats = TrainingStats()
        if config.validation_dataset_path is not None:
            self.validation_flag = True

        # create dataloaders
        self.train_dataset = ActorDataset(config.train_dataset_path)
        self.train_dataloader = DataLoader(
            self.train_dataset, batch_size=config.batch_size
        )
        if self.validation_flag:
            self.eval_dataset = ActorDataset(config.validation_dataset_path)
            self.validation_dataloader = DataLoader(
                self.eval_dataset, batch_size=config.batch_size
            )

        # initialize deepspeed
        self.model_engine = None
        if config.deepspeed_enable is True:
            if config.deepspeed_config_path is None:
                raise ValueError(
                    "DeepSpeed config path is None, but deepspeed is enabled"
                )
            if os.path.exists(config.deepspeed_config_path) is False:
                raise ValueError(
                    f"DeepSpeed config path {config.deepspeed_config_path}"
                    f"does not exist"
                )
            (
                self.model_engine,
                self.optimizer,
                self.train_dataloader,
                _,
            ) = deepspeed.initialize(
                args=None,
                model=self.model,
                model_parameters=self.model.parameters(),
                training_data=self.train_dataloader,
                config=self.config.deepspeed_config_path,
            )

    @beartype
    def save_checkpoint(
        self,
        current_epoch: int,
        current_step: int,
        max_epochs: int,
        max_steps: int,
    ) -> None:

        print(
            f"Saving checkpoint for epoch {current_epoch + 1}, "
            f"step {current_step + 1} ..."
        )
        model_folder, model_name, path = ModelLoader.get_model_path(
            config=self.config,
            is_checkpoint=True,
            current_epoch=current_epoch,
            current_step=current_step,
            max_epochs=max_epochs,
            max_steps=max_steps,
        )
        if os.path.exists(path):
            os.remove(path)
        torch.save(
            {
                "state_dict": self.model.state_dict(),
                "optim_state_dict": self.optimizer.state_dict(),
                "training_stats": self.training_stats,
                "epoch": current_epoch,
                "step": current_step,
            },
            path,
        )

    @beartype
    def load_checkpoint(
        self,
    ) -> Tuple[int, int]:
        """Load a checkpoint from the model folder"""

        print("Looking for checkpoints...")
        path = ModelLoader.check_model_path(
            config=self.config,
            is_checkpoint=True,
            current_epoch=None,
        )
        if path is not None:
            print("Loading ...")
            checkpoint = torch.load(path)
            epoch = checkpoint["epoch"]
            self.model.load_state_dict(checkpoint["state_dict"])
            self.optimizer.load_state_dict(checkpoint["optim_state_dict"])
            self.trainign_stats = checkpoint["training_stats"]
            step = checkpoint["step"]
            return epoch, step + 1  # return the next episode to train
        return 0, 0

    def train(
        self,
    ) -> None:
        print("Start Actor Model Pretraining")

        # get config parameters
        batch_size = self.config.batch_size
        epochs = self.config.epochs
        device = self.config.device
        checkpoint_steps = self.config.checkpoint_steps

        # compute the number of iterations
        n_iter = int(len(self.train_dataset) / batch_size)

        # load model_checkpoint
        start_epoch, start_step = self.load_checkpoint()

        # counter for the checkpoint
        cnt_checkpoint = 1

        # traing loop
        for epoch in range(start_epoch, epochs):
            self.model.train()
            for i, input_output in enumerate(self.train_dataloader):
                if i < start_step:
                    continue
                with torch.no_grad():
                    input_output_tokenized = self.model.tokenizer(
                        input_output,
                        return_tensors="pt",
                        padding=True,
                    )
                    training_output = input_output_tokenized["input_ids"][
                        :, 1:
                    ]
                    training_input = input_output_tokenized["input_ids"][
                        :, :-1
                    ]
                    attention_mask = input_output_tokenized["attention_mask"][
                        :, :-1
                    ]
                    training_output = training_output.to(device)
                    training_input = training_input.to(device)
                    attention_mask = attention_mask.to(device)

                # forward pass
                if self.config.deepspeed_enable:
                    est_output = self.model_engine(
                        training_input, attention_mask
                    )
                else:
                    est_output = self.model(training_input, attention_mask)
                est_output = rearrange(est_output, "b s v -> (b s) v")
                training_output = rearrange(training_output, "b s -> (b s)")
                loss = self.loss_function(est_output, training_output)
                self.training_stats.training_loss.append(loss.item())

                # backward pass
                if self.config.deepspeed_enable:
                    self.model_engine.backward(loss)
                    self.model_engine.step()
                else:
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                # print progress
                if i % self.config.iteration_per_print == 0:
                    print(
                        f"Epoch: {epoch+1}/{epochs}, "
                        f"Iteration: {i+1}/{n_iter}, "
                        f"Training Loss: {loss}"
                    )
                # save checkpoint periodically
                if cnt_checkpoint % checkpoint_steps == 0:
                    self.save_checkpoint(epoch, i, epochs, n_iter)
                    cnt_checkpoint = 1
                else:
                    cnt_checkpoint += 1

            if self.validation_flag:
                self.model.eval()
                for i, input_output in enumerate(self.validation_dataloader):
                    input_output_tokenized = self.model.tokenizer(
                        input_output, return_tensors="pt", padding=True
                    )
                    validation_output = input_output_tokenized["input_ids"][
                        :, 1:
                    ]
                    validation_input = input_output_tokenized["input_ids"][
                        :, :-1
                    ]
                    attention_mask = input_output_tokenized["attention_mask"][
                        :, :-1
                    ]

                    # forward pass
                    est_output = self.model.forward(
                        validation_input, attention_mask
                    )
                    validation_output = rearrange(
                        validation_output, "b s -> (b s)"
                    )
                    est_output = rearrange(est_output, "b s v -> (b s) v")
                    loss = self.loss_function(est_output, validation_output)
                    self.training_stats.validation_loss.append(loss.item())
                    # print progress
                    if i % self.config.iteration_per_print == 0:
                        print(
                            f"Epoch: {epoch+1}/{epochs}, "
                            f"Iteration: {i+1}/{n_iter}, "
                            f"Validation Loss: {loss}"
                        )
        self.model.save()
        print("Training Finished ")

In [ ]:
def generate_test_contrct(path, ratio): 
  col = ['Target']
  f = open(path, "r")
  file = f.read()
  test_df = pd.DataFrame([file], columns=col)
  program_length = len(test_df['Target'][0].split())
  prompt_ratio = ratio
  prompt_length = int(prompt_ratio * program_length)
  return test_df, prompt_length

def truncate_test(df, prompt_length):
  copy_1 = df.copy(deep=True)
  copy_2 = df.copy(deep=True)
  #true
  a = copy_1['Target'].str.split().str[-prompt_length:].apply(' '.join)[0]
  #masked out program 
  b = copy_2['Target'].str.split().str[:-prompt_length].apply(' '.join)[0]
  return a, b

In [ ]:
test_df, n = generate_test_contrct('/content/drive/MyDrive/experiments/baseline benchmark/Replica.sol', 0.95)
truth, prompt = truncate_test(test_df, n)
col = ['Target']
prompt_df = pd.DataFrame([prompt], columns=col)
gpt_out = text_generation(trained_model, tokenizer, prompt_df)
print('this is the GPT prompt without T5: \n', prompt_df['Target'][0])
print('this is the GPT prediction without T5: \n', gpt_out)